In [1]:
from bw_temporalis import easy_timedelta_distribution, easy_datetime_distribution, TemporalisLCA, Timeline, TemporalDistribution
from bw_temporalis.lcia import characterize_methane, characterize_co2
import bw2data as bd
import bw2calc as bc
import bw_graph_tools as graph
import numpy as np
import pandas as pd

In [2]:
bd.projects.set_current("📊📈💎🕤🗓️")

In [3]:
bd.Database('temporalis-example').write({
    ('temporalis-example', "CO2"): {
        "type": "emission",
        "name": "carbon dioxide",
        "temporalis code": "co2",
    },
    ('temporalis-example', "CH4"): {
        "type": "emission",
        "name": "methane",
        "temporalis code": "ch4",
    },
    ('temporalis-example', 'Functional Unit'): {
        'name': 'Functional Unit',
        'exchanges': [
            {
                'amount': 5,
                'input': ('temporalis-example', 'EOL'),
                'temporal_distribution': easy_timedelta_distribution(
                    start=0,
                    end=4, # Range includes both start and end
                    resolution="Y",  # M for months, Y for years, etc.
                    steps=5,
                ),
                'type': 'technosphere'
            },
        ],
    },
    ('temporalis-example', 'EOL'): {
        'exchanges': [
            {
                'amount': 0.8,
                'input': ('temporalis-example', 'Waste'),
                'type': 'technosphere'
            },
            {
                'amount': 0.2,
                'input': ('temporalis-example', 'Landfill'),
                'type': 'technosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Use'),
                'type': 'technosphere'
            },
        ],
        'name': 'EOL',
        'type': 'process'
    },
    ('temporalis-example', 'Use'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Production'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-4], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Use',
    },
    ('temporalis-example', 'Production'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Transport'),
                'temporal_distribution': TemporalDistribution(
                    np.array([200],dtype='timedelta64[D]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Production',
        'type': 'process'
    },
    ('temporalis-example', 'Transport'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Sawmill'),
                'type': 'technosphere'
            },
            {
                'amount': 0.1,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
        ],
        'name': 'Production',
        'type': 'process'
    },
    ('temporalis-example', 'Sawmill'): {
        'exchanges': [
            {
                'amount': 1.2,
                'input': ('temporalis-example', 'Forest'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-14], dtype='timedelta64[M]'),
                    np.array([1.0])
                ),
                'type': 'technosphere'
            },
            {
                'amount': 0.1,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
        ],
        'name': 'Sawmill',
        'type': 'process'
    },
    ('temporalis-example', 'Forest'): {
        'exchanges': [
            {
                'amount': -.1 * 6,
                'input': ('temporalis-example', 'CO2'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-4, -3, 0, 1, 2, 5], dtype='timedelta64[Y]'),
                    np.ones(6) * (1/6)
                ),
                'type': 'biosphere'
            },
            {
                'amount': 1.5,
                'input': ('temporalis-example', 'Thinning'),
                'temporal_distribution': TemporalDistribution(
                    np.array([-3, 0, 1], dtype='timedelta64[Y]'),
                    np.ones(3) * 1/3
                ),
                'type': 'technosphere'
            },
        ],
        'name': 'Forest',
    },
    ('temporalis-example', 'Thinning'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Thinning'),
                'type': 'production'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - thinnings'),
                'type': 'production'
            },
        ],
        'name': 'Thinning',
        'type': 'process'
    },
    ('temporalis-example', 'Landfill'): {
        'exchanges': [
            {
                'amount': 0.1,
                'input': ('temporalis-example', 'CH4'),
                'temporal_distribution': TemporalDistribution(
                    np.array([10, 20, 60, 100], dtype='timedelta64[M]'),
                    np.ones(4) * 1/4
                ),
                'type': 'biosphere'
            },
        ],
        'name': 'Landfill',
        'type': 'process'
    },
    ('temporalis-example', 'Waste'): {
        'exchanges': [
            {
                'amount': 1,
                'input': ('temporalis-example', 'Waste'),
                'type': 'production'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - waste'),
                'type': 'production'
            },
        ],
        'name': 'Waste',
        'type': 'process'
    },
    ('temporalis-example', 'Avoided impact - waste'): {
        'exchanges': [
            {
                'amount': -0.4,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - waste'),
                'type': 'production'
            },
        ],
        'name': 'Avoided impact - waste',
        'type': 'process'
    },
    ('temporalis-example', 'Avoided impact - thinnings'): {
        'exchanges': [
            {
                'amount': -0.3,
                'input': ('temporalis-example', 'CO2'),
                'type': 'biosphere'
            },
            {
                'amount': 1,
                'input': ('temporalis-example', 'Avoided impact - thinnings'),
                'type': 'production'
            },
        ],
        'name': 'Avoided impact - thinnings',
        'type': 'process'
    }
})

Not able to determine geocollections for all datasets. This database is not ready for regionalization.


100%|██████████████████████████████████████████████████████████████████████████████████████████| 14/14 [00:00<?, ?it/s]

Vacuuming database 


In [4]:
bd.Method(("GWP", "example")).write([
    (('temporalis-example', "CO2"), 1),
    (('temporalis-example', "CH4"), 25),
])

In [5]:
lca = bc.LCA({('temporalis-example', 'EOL'): 1}, ("GWP", "example"))
lca.lci()
lca.lcia()

# Custom `EdgeExtractor` class

In [6]:
from dataclasses import dataclass
from heapq import heappop, heappush
from typing import Callable, List

from bw_temporalis import TemporalisLCA, TemporalDistribution


@dataclass
class Edge:
    """
    Class for storing a temporal edge with source and target.

    Leaf edges link to a source process which is a leaf in
    our graph traversal (either through cutoff or a filter
    function).

    Attributes
    ----------
    distribution : TemporalDistribution
    leaf : bool
    consumer : int
    producer : int

    """

    distribution: TemporalDistribution
    leaf: bool
    consumer: int
    producer: int


class EdgeExtracter(TemporalisLCA):
    def __init__(self, *args, edge_filter_function: Callable = None, **kwargs):
        super().__init__(*args, **kwargs)
        if edge_filter_function:
            self.edge_ff = edge_filter_function
        else:
            self.edge_ff = lambda x: False

    def build_edge_timeline(self, node_timeline: bool | None = False) -> List:
        heap = []
        timeline = []

        for edge in self.edge_mapping[self.unique_id]:
            node = self.nodes[edge.producer_unique_id]
            heappush(
                heap,
                (
                    1 / node.cumulative_score,
                    self.t0 * edge.amount,
                    node,
                ),
            )
            timeline.append(
                Edge(
                    distribution=self.t0 * edge.amount,
                    leaf=False,
                    consumer=self.unique_id,
                    producer=node.activity_datapackage_id,
                )
            )

        while heap:
            _, td, node = heappop(heap)

            for edge in self.edge_mapping[node.unique_id]:
                row_id = self.nodes[edge.producer_unique_id].activity_datapackage_id
                col_id = node.activity_datapackage_id
                exchange = self.get_technosphere_exchange(
                    input_id=row_id,
                    output_id=col_id,
                )
                value = (
                    self._exchange_value(
                        exchange=exchange,
                        row_id=row_id,
                        col_id=col_id,
                        matrix_label="technosphere_matrix",
                    )
                    / node.reference_product_production_amount
                )
                producer = self.nodes[edge.producer_unique_id]
                leaf = self.edge_ff(row_id)

                distribution = (td * value).simplify()
                timeline.append(
                    Edge(
                        distribution=distribution,
                        leaf=leaf,
                        consumer=node.activity_datapackage_id,
                        producer=producer.activity_datapackage_id,
                    )
                )
                if not leaf:
                    heappush(
                        heap,
                        (
                            1 / node.cumulative_score,
                            distribution,
                            producer,
                        ),
                    )
        return timeline

In [7]:
SKIPPABLE = [
    bd.get_node(code=code).id 
    for code in ('Avoided impact - waste', 'Landfill', 'Thinning')
]

def filter_function(database_id: int) -> bool:
    return database_id in SKIPPABLE

In [8]:
eelca = EdgeExtracter(lca, edge_filter_function=filter_function)

Starting graph traversal
Calculation count: 10


In [9]:
tl = eelca.build_edge_timeline()

In [10]:
tl

[Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=-1, producer=4),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=4, producer=5),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 0.2, leaf=True, consumer=4, producer=11),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 0.8, leaf=False, consumer=4, producer=12),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 0.8, leaf=True, consumer=12, producer=13),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=5, producer=6),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=6, producer=7),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 1, leaf=False, consumer=7, producer=8),
 Edge(distribution=TemporalDistribution instance with 1 values and total: 1.2, 

In [58]:
#group flows from same producer to consumer in the same year

edges_dict_list = [{"datetime": edge.distribution.date, 'amount': edge.distribution.amount, 'producer': edge.producer, 'consumer': edge.consumer, "leaf": edge.leaf} for edge in tl]
edges_dataframe = pd.DataFrame(edges_dict_list)
edges_dataframe = edges_dataframe.explode(['datetime', "amount"])
edges_dataframe['year'] = edges_dataframe['datetime'].apply(lambda x: x.year)
edge_dataframe = edges_dataframe.loc[:, "amount":].groupby(['year', 'producer', 'consumer']).sum().reset_index()
print(edge_dataframe) 


    year  producer  consumer amount  leaf
0   2019        10         9    0.6     1
1   2022         9         8    1.2     0
2   2022        10         9    0.6     1
3   2023         4        -1    1.0     0
4   2023         5         4    1.0     0
5   2023         6         5    1.0     0
6   2023         7         6    1.0     0
7   2023         8         7    1.0     0
8   2023        10         9    0.6     1
9   2023        11         4    0.2     1
10  2023        12         4    0.8     0
11  2023        13        12    0.8     1


Compare with basic timeline:

In [11]:
tlca = TemporalisLCA(lca)
tl_original = tlca.build_timeline()
tl_original.data

Starting graph traversal
Calculation count: 10


[FlowTD(distribution=TemporalDistribution instance with 4 values and total: 0.02, flow=2, activity=11),
 FlowTD(distribution=TemporalDistribution instance with 1 values and total: -0.32, flow=1, activity=13),
 FlowTD(distribution=TemporalDistribution instance with 1 values and total: 0.1, flow=1, activity=7),
 FlowTD(distribution=TemporalDistribution instance with 1 values and total: 0.1, flow=1, activity=8),
 FlowTD(distribution=TemporalDistribution instance with 6 values and total: -0.72, flow=1, activity=9),
 FlowTD(distribution=TemporalDistribution instance with 3 values and total: -0.54, flow=1, activity=14)]